In [8]:
import yfinance as yf
import pandas as pd

# Fetch daily S&P 500 data
sp500_data = yf.download('^GSPC', start='2000-01-01', end='2024-01-01')

# Calculate additional features for S&P 500 data
sp500_data['daily_return'] = sp500_data['Close'].pct_change() #might need
sp500_data['volatility'] = sp500_data['daily_return'].rolling(window=20).std()

###sp500_data['ma_200'] = sp500_data['Close'].rolling(window=200).mean() # for long term


# Calculate the 200-day moving return
sp500_data['200_day_return'] = sp500_data['Close'].pct_change(periods=200)

[*********************100%%**********************]  1 of 1 completed


In [13]:
sp500 = sp500_data.dropna()
sp500.shape

(5837, 9)

In [9]:
import pandas_datareader.data as pdr
import yfinance as yf
import pandas as pd

# Fetch economic indicators from FRED
gdp = pdr.get_data_fred('GDP', start='2000-01-01', end='2024-01-01') #quaterly

inflation = pdr.get_data_fred('CPIAUCSL', start='2000-01-01', end='2024-01-01')  # CPI for inflation
unemployment = pdr.get_data_fred('UNRATE', start='2000-01-01', end='2024-01-01')
interest_rate = pdr.get_data_fred('FEDFUNDS', start='2000-01-01', end='2024-01-01')

# Rename columns for clarity
gdp.rename(columns={'GDP': 'GDP'}, inplace=True)
inflation.rename(columns={'CPIAUCSL': 'Inflation'}, inplace=True)
unemployment.rename(columns={'UNRATE': 'Unemployment'}, inplace=True)
interest_rate.rename(columns={'FEDFUNDS': 'Interest_Rate'}, inplace=True)

In [10]:
GDP = gdp.resample('D').ffill()
GDP.loc['2020-03-28':'2020-04-01'] 

,GDP
DATE,
2020-03-28,21706.513
2020-03-29,21706.513
2020-03-30,21706.513
2020-03-31,21706.513
2020-04-01,19913.143


In [11]:
Inflation = inflation.resample('D').ffill()
Unemployment = unemployment.resample('D').ffill()
Interest_rate = interest_rate.resample('D').ffill()

In [14]:
merged_data = sp500.join([Inflation, Unemployment, Interest_rate], how='left')
merged_data.head()

,Open,High,Low,Close,Adj Close,Volume,daily_return,volatility,200_day_return,Inflation,Unemployment,Interest_Rate
Date,,,,,,,,,,,,
2000-10-17,1374.619995,1380.989990,1342.339966,1349.969971,1349.969971,1.161500e+09,-0.017932,0.013576,-0.072326,173.9,3.9,6.51
2000-10-18,1349.969971,1356.650024,1305.790039,1342.130005,1342.130005,1.441700e+09,-0.005808,0.013576,-0.040938,173.9,3.9,6.51
2000-10-19,1342.130005,1389.930054,1342.130005,1388.760010,1388.760010,1.297900e+09,0.034743,0.016088,-0.009521,173.9,3.9,6.51
2000-10-20,1388.760010,1408.469971,1382.189941,1396.930054,1396.930054,1.177400e+09,0.005883,0.016181,-0.004646,173.9,3.9,6.51
2000-10-23,1396.930054,1406.959961,1387.750000,1395.780029,1395.780029,1.046800e+09,-0.000823,0.016139,-0.031697,173.9,3.9,6.51


In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# y
y = np.where(merged_data['Close'].shift(-1) > merged_data['Close'], 1, -1)
# feature
X = merged_data.drop(columns=['Close', 'daily_return', '200_day_return']).shift(1)


y = y[:-1]
X = X.dropna()


y = y[-len(X):]

# Check the shape of 'y' and 'X'
print(y.shape, X.shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, shuffle=False)

# Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Generate a classification report
report = classification_report(y_test, y_pred)
print(report)


(5836,) (5836, 9)
              precision    recall  f1-score   support

          -1       0.46      0.73      0.57      1878
           1       0.54      0.27      0.36      2208

    accuracy                           0.48      4086
   macro avg       0.50      0.50      0.46      4086
weighted avg       0.50      0.48      0.45      4086

